In [ ]:
# !pip install polars
# !pip install optuna
# !pip install imblearn
# !pip install lightgbm
# !pip install catboost
# !pip install xgboost
# !pip install eli5
# !pip install scikit-learn==1.2.0

### Notes

- base code is taken from [here](https://www.kaggle.com/code/greysky/isic-2024-only-tabular-data)

In [29]:
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import plotly.graph_objects as go

from typing import List, Tuple, Dict, Optional
from pydantic import BaseModel

from tqdm import tqdm

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score, StratifiedGroupKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, silhouette_score
from sklearn.ensemble import VotingClassifier
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

from catboost import CatBoostClassifier, Pool

from matplotlib import pyplot as plt

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from PIL import Image

from scipy.stats import ttest_ind, ttest_1samp, ttest_rel

from eli5.permutation_importance import get_score_importances

import optuna
from optuna.trial import TrialState

import warnings
warnings.filterwarnings('ignore')

In [2]:
root = Path('../data/original')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42


num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]

# age_sex_norm_columns_raw = [
#         'tbp_lv_H', 'age_approx','hue_contrast',
#        'age_normalized_nevi_confidence_2', 'tbp_lv_deltaB',
       # 'color_uniformity', 'tbp_lv_z',
       # # 'clin_size_long_diam_mm', 'tbp_lv_y',
       # # 'position_distance_3d', 'hue_contrast',
       # # 'tbp_lv_stdLExt', 'mean_hue_difference',
       # # 'age_normalized_nevi_confidence',
       # # 'lesion_visibility_score',
       # # 'position_distance_3d',
       # 'tbp_lv_minorAxisMM', 'tbp_lv_Hext']
# age_sex_norm_columns = [f'{col}_age_sex_norm' for col in age_sex_norm_columns_raw]


# attribution_norm_cols = [f'{col}_attribution_norm' for col in attribution_norm_cols_row]
special_cols = ['count_per_patient', "tbp_lv_areaMM2_patient", "tbp_lv_areaMM2_bp"] #'count_per_patient_bp'
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols 
# + age_sex_norm_columns
# + attribution_norm_cols

# extra_features = ['0_channel_mean_patient', '1_channel_mean_patient', '2_channel_mean_patient', '0_channel_r', '1_channel_r', '2_channel_r']
# feature_cols += extra_features

In [3]:
def set_seed(random_seed):
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

In [4]:
train_df_raw = pd.read_csv(train_path)

/tmp/ipykernel_3471571/2662678883.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_raw = pd.read_csv(train_path)


In [5]:
train_df_raw.head(2)

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455


In [6]:
def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        # .with_columns(
        #     ((pl.col(col) - pl.col(col).mean().over(['attribution'])) / (pl.col(col).std().over(['attribution']) + err)).alias(f'{col}_attribution_norm') for col in attribution_norm_cols_row
        # )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )

        .with_columns(
            tbp_lv_areaMM2_patient = pl.col('tbp_lv_areaMM2').sum().over('patient_id'),
        )
        .with_columns(
            tbp_lv_areaMM2_bp = pl.col('tbp_lv_areaMM2').sum().over(['patient_id', 'anatom_site_general']),
        )


        # .with_columns(
        #     tbp_lv_minorAxisMM_age_norm = pl.col('tbp_lv_minorAxisMM') / pl.col('tbp_lv_minorAxisMM').mean().over(['age_approx', 'sex']),
        # )
        # .with_columns(
        #     ((pl.col(col) / pl.col(col).mean().over(['age_approx', 'sex']))).alias(f'{col}_age_sex_norm') for col in age_sex_norm_columns_raw
        # )

        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

In [7]:
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [8]:
def custom_metric_raw(y_hat, y_true):
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc


def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    partial_auc = custom_metric_raw(y_hat, y_true)
    return partial_auc
    

In [9]:
def get_lof_score(df_train, top_lof_features, output_col_name="of", lof_column='patient_id'):
    scalled_array = StandardScaler().fit_transform(df_train[top_lof_features])
    outiler_factors = []
    for tmp_col_v in tqdm(df_train[lof_column].unique()):
        mask = (df_train[lof_column] == tmp_col_v).values
        if sum(mask) < 3:
            continue
        patient_emb = scalled_array[mask]
        clf = LocalOutlierFactor(n_neighbors=min(sum(mask), 30))
        clf.fit_predict(patient_emb)
        
    
        outiler_factors.append(pd.DataFrame(
            {"isic_id": df_train[mask].index.values,
            output_col_name: clf.negative_outlier_factor_}))
    
    outiler_factors = pd.concat(outiler_factors).reset_index(drop=True)
    
    df_train = df_train.merge(outiler_factors.set_index('isic_id'), how="left", left_index=True, right_index=True)
    df_train[output_col_name] = df_train[output_col_name].fillna(-1)
    return df_train

In [10]:
top_lof_features = [
        'tbp_lv_H','hue_contrast',
       'age_normalized_nevi_confidence_2', 'tbp_lv_deltaB',
       'color_uniformity', 'tbp_lv_z',
       'clin_size_long_diam_mm', 'tbp_lv_y',
       'position_distance_3d', 'hue_contrast',
       'tbp_lv_stdLExt', 'mean_hue_difference',
       'age_normalized_nevi_confidence',
       'lesion_visibility_score',
       'position_distance_3d',
       'tbp_lv_minorAxisMM', 'tbp_lv_Hext']

In [11]:
df_train = read_data(train_path)
df_test = read_data(test_path)
df_subm = pd.read_csv(subm_path, index_col=id_col)

df_train, df_test = preprocess(df_train, df_test)

In [12]:
df_train = get_lof_score(df_train, top_lof_features, output_col_name="of", lof_column='patient_id')
feature_cols += ['of']

100%|██████████| 1042/1042 [05:07<00:00,  3.38it/s]


In [13]:
### get predictions from model based on external data
old_data_model_preds = pd.read_parquet('../data/artifacts/old_data_model_forecast_large.parquet') #old_data_model_forecast old_data_model_forecast
df_train = df_train.merge(old_data_model_preds, how="left", on=["isic_id"])

feature_cols += ['old_set_0', 'old_set_1', 'old_set_2']

df_train = df_train.merge(
    df_train.groupby("patient_id").agg(**{
        "old_set_0_m": pd.NamedAgg('old_set_0', 'mean'),
        "old_set_1_m": pd.NamedAgg('old_set_1', 'mean'),
        "old_set_2_m": pd.NamedAgg('old_set_2', 'mean')
    }).reset_index(), how="left", on=["patient_id"])

df_train['old_set_0_m'] = df_train['old_set_0'] / df_train['old_set_0_m']
df_train['old_set_1_m'] = df_train['old_set_1'] / df_train['old_set_1_m']
df_train['old_set_2_m'] = df_train['old_set_2'] / df_train['old_set_2_m']

feature_cols += ['old_set_0_m', 'old_set_1_m', 'old_set_2_m']

In [18]:
###get oof model predictions (eva)
oof_forecasts = pd.read_parquet('../data/artifacts/oof_forecasts_eva_base.parquet')

# tmp_predictions_all__pr
df_train = df_train.merge(
    oof_forecasts[['isic_id', 'tmp_predictions_all__pr']].rename(columns={
        'tmp_predictions_all__pr': 'oof_eva_score'
    }), how="left", on=['isic_id'])


df_train = df_train.merge(
    df_train.groupby("patient_id").agg(**{
        "oof_eva_score_m": pd.NamedAgg('oof_eva_score', 'mean')
    }).reset_index(), how="left", on=["patient_id"])

df_train['oof_eva_score_m'] = df_train['oof_eva_score'] / df_train['oof_eva_score_m']

feature_cols += ['oof_eva_score', 'oof_eva_score_m']

In [23]:
###get oof model predictions (edgenext)
oof_forecasts = pd.read_parquet('../data/artifacts/oof_forecasts_edgenext_base.parquet') # oof_forecasts_edgenext_base

# tmp_predictions_all__pr
df_train = df_train.merge(
    oof_forecasts[['isic_id', 'tmp_predictions_all__pr']].rename(columns={
        'tmp_predictions_all__pr': 'oof_edgenext_score'
    }), how="left", on=['isic_id'])

df_train = df_train.merge(
    df_train.groupby("patient_id").agg(**{
        "oof_edgenext_score_m": pd.NamedAgg('oof_edgenext_score', 'mean')
    }).reset_index(), how="left", on=["patient_id"])

df_train['oof_edgenext_score_m'] = df_train['oof_edgenext_score'] / df_train['oof_edgenext_score_m']

feature_cols += ['oof_edgenext_score', 'oof_edgenext_score_m']

In [24]:
###get oof based aggs
df_train = df_train.merge(
    df_train.groupby(['attribution', 'tbp_lv_location', 'age_approx']).agg(**{
        "oof_edgenext_score_mean_tmp": pd.NamedAgg('oof_edgenext_score', 'mean'),
        "oof_edgenext_score_std_tmp": pd.NamedAgg('oof_edgenext_score', 'std'),
        "oof_eva_score_mean_tmp": pd.NamedAgg('oof_eva_score', 'mean'),
        "oof_eva_score_std_tmp": pd.NamedAgg('oof_eva_score', 'std')
    }).reset_index(), how="left", on=['attribution', 'tbp_lv_location', 'age_approx'])

df_train["oof_edgenext_score_mean_tmp"] = (
    df_train["oof_edgenext_score"] - df_train["oof_edgenext_score_mean_tmp"]) / df_train["oof_edgenext_score_std_tmp"]
df_train["oof_eva_score_mean_tmp"] = (
    df_train["oof_eva_score"] - df_train["oof_eva_score_mean_tmp"]) / df_train["oof_eva_score_std_tmp"]

feature_cols += ['oof_edgenext_score_mean_tmp', 'oof_eva_score_mean_tmp']

/tmp/ipykernel_3471571/3335836358.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train.groupby(['attribution', 'tbp_lv_location', 'age_approx']).agg(**{


In [25]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_train[top_lof_features])

np.random.seed(42)
kmeans = KMeans(n_clusters=29, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [26]:
columns_for_cluster_culculations = [
    "tbp_lv_minorAxisMM", "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_symm_2axis",
    "tbp_lv_radial_color_std_max", "lesion_visibility_score", "mean_hue_difference", "clin_size_long_diam_mm",
    "color_uniformity", "hue_contrast", "tbp_lv_H", "oof_eva_score", "oof_edgenext_score",
    'tbp_lv_Aext', 'tbp_lv_Bext', 'tbp_lv_Cext', 'tbp_lv_Lext', 
    'tbp_lv_deltaLBnorm', 'tbp_lv_stdLExt', 'position_distance_3d', 'color_contrast_index',
    'age_normalized_nevi_confidence_2', 'volume_approximation_3d', 'age_approx_patient_norm', 'tbp_lv_H_patient_norm',
    'tbp_lv_Hext_patient_norm', 'tbp_lv_L_patient_norm', 'tbp_lv_stdLExt_patient_norm', 'tbp_lv_symm_2axis_angle_patient_norm',
    'tbp_lv_x_patient_norm', 'tbp_lv_y_patient_norm', 'tbp_lv_z_patient_norm', 'lesion_size_ratio_patient_norm', 'hue_contrast_patient_norm',
    'color_uniformity_patient_norm', 'position_distance_3d_patient_norm', 'size_age_interaction_patient_norm', 'mean_hue_difference_patient_norm',
    'std_dev_contrast_patient_norm', 'lesion_orientation_3d_patient_norm', 
    'volume_approximation_3d_patient_norm', 'color_range_patient_norm', 'tbp_lv_areaMM2_patient', 'tbp_lv_areaMM2_bp', 'of',
    'old_set_0_m', 'old_set_1_m', 'oof_eva_score_m', 'oof_edgenext_score_m', 'oof_edgenext_score_mean_tmp', 'oof_eva_score_mean_tmp',
]

df_train['cluster_labels'] = cluster_labels
agg_config = {
    f"{i}__mean": pd.NamedAgg(i, "mean")
        for i in columns_for_cluster_culculations
}
agg_config.update({
    f"{i}__std": pd.NamedAgg(i, "std")
        for i in columns_for_cluster_culculations
})
df_train = df_train.merge(
    df_train.groupby("cluster_labels").agg(**agg_config), how="left", on="cluster_labels")

for col in columns_for_cluster_culculations:
    df_train[f"{col}__cluster"] = (df_train[col] - df_train[f'{col}__mean']) / df_train[f'{col}__std']
    feature_cols += [f"{col}__cluster"]

In [27]:
def run_model_old(cb_params, reduce=True, n_rounds=10, columns_to_drop=None) -> float:
    metric_list = []
    columns_to_drop = [] if columns_to_drop is None else columns_to_drop
    for random_seed in range(1, n_rounds):
        tsp = StratifiedGroupKFold(5, shuffle=True, random_state=random_seed)
        metrics_ev_df = []
        test_forecast = []
        val_forecast = []
        for fold_n, (train_index, val_index) in enumerate(tsp.split(df_train, y=df_train.target, groups=df_train[group_col])):
            train_slice_x = df_train.iloc[train_index][[i for i in feature_cols if i not in columns_to_drop]].reset_index(drop=True)
            val_slice_x = df_train.iloc[val_index][[i for i in feature_cols if i not in columns_to_drop]].reset_index(drop=True)
            
            train_slice_y = df_train.iloc[train_index]['target'].reset_index(drop=True)
            val_slice_y = df_train.iloc[val_index]['target'].reset_index(drop=True)
        
            cb_model = Pipeline([
                ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
                ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio , random_state=seed)),
                ('classifier', cb.CatBoostClassifier(**cb_params)),
            ])
            
            cb_model.fit(train_slice_x, train_slice_y)
            preds_cb = cb_model.predict_proba(val_slice_x)[:, 1]
            metric = custom_metric_raw(preds_cb, val_slice_y.values)
            metric_list.append(metric)

    if reduce:
        return np.mean(metric_list)
    else:
        return metric_list

In [28]:
class ModelConfigCB(BaseModel):
    iterations: int = 1000
    learning_rate: float = 0.06936242010150652
    l2_leaf_reg: float = 6.216113851699493
    loss_function: str = "Logloss"
    bagging_temperature: float = 1
    random_seed: int = seed
    border_count: int = 128
    grow_policy: str = "SymmetricTree" #Depthwise Lossguide
    min_data_in_leaf: int = 24
    depth: int  = 7
    do_sample: bool = True

model_config_cb = ModelConfigCB()

In [31]:
def optimise_catboost(trial: optuna.Trial, test=False, reduce=True, columns_to_drop=None, n_rounds=10, cat_cols=None, get_fe=False) -> float:
    cat_cols = [] if cat_cols is None else cat_cols
    columns_to_drop = [] if columns_to_drop is None else columns_to_drop
    model_config_cb = ModelConfigCB(
        iterations = 2000,
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.08) if not test else trial.get('learning_rate'),
        l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1, 20) if not test else trial.get('l2_leaf_reg'),
        random_strength = trial.suggest_float('random_strength', 0, 5) if not test else trial.get('random_strength'),
        loss_function = "Logloss",
        depth = trial.suggest_int('depth', 2, 8) if not test else trial.get('depth'),
        bagging_temperature = trial.suggest_float('bagging_temperature', 0, 10) if not test else trial.get('bagging_temperature'),
        border_count = trial.suggest_categorical('border_count', [128, 256]) if not test else trial.get('border_count'),
        grow_policy = trial.suggest_categorical('grow_policy', ["SymmetricTree", "Depthwise", "Lossguide"]) if not test else trial.get('grow_policy'),
        random_seed=42,
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 8, 40) if not test else trial.get('min_data_in_leaf'),
    )

    metric_list = []
    models = []
    all_val_df_tmp = []
    score_decreases_list = []
    for random_seed in range(1, n_rounds):
        tsp = StratifiedGroupKFold(5, shuffle=True, random_state=random_seed)
        metrics_ev_df = []
        test_forecast = []
        val_forecast = []
        for fold_n, (train_index, val_index) in enumerate(tsp.split(df_train, y=df_train.target, groups=df_train[group_col])):
            train_slice_x = df_train.iloc[train_index][[i for i in feature_cols if i not in columns_to_drop]].reset_index(drop=True)
            val_slice_x = df_train.iloc[val_index][[i for i in feature_cols if i not in columns_to_drop]].reset_index(drop=True)

            train_slice_y = df_train.iloc[train_index]['target'].reset_index(drop=True)
            val_slice_y = df_train.iloc[val_index]['target'].reset_index(drop=True)
            
            if model_config_cb.do_sample:
                cb_model = Pipeline([
                    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=random_seed)),
                    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio , random_state=random_seed)),
                ])
                
                train_slice_x, train_slice_y = cb_model.fit_resample(train_slice_x, train_slice_y)
            
        
            clf_catboost = CatBoostClassifier(
                loss_function=model_config_cb.loss_function,
                eval_metric='AUC',
                task_type='GPU',
                learning_rate=model_config_cb.learning_rate,
                od_wait=100,
                random_state=random_seed,
                depth=model_config_cb.depth,
                l2_leaf_reg=model_config_cb.l2_leaf_reg,
                min_data_in_leaf=model_config_cb.min_data_in_leaf,
                bagging_temperature=model_config_cb.bagging_temperature,
                border_count=model_config_cb.border_count,
                grow_policy=model_config_cb.grow_policy, 
                devices='0',
                iterations=model_config_cb.iterations,
            )
        
            train_pool = Pool(train_slice_x, train_slice_y.values, cat_features=cat_cols) 
            val_pool = Pool(val_slice_x, val_slice_y.values, cat_features=cat_cols) 
        
            clf_catboost.fit(train_pool, eval_set=val_pool,verbose=False)
            preds_cb = clf_catboost.predict_proba(val_slice_x)[:, 1]
            metric = custom_metric_raw(preds_cb, val_slice_y.values)
            metric_list.append(metric)
            models.append(clf_catboost)
            
            
            
            # def make_score(clf, columns_names):
            #     def score(X, y):
            #         predictions_tmp = clf.predict_proba(pd.DataFrame(X, columns=columns_names), thread_count=-1)[:, 1]
            #         return custom_metric_raw(predictions_tmp, y)
            #     return score

            if get_fe:
                score_decreases = permutation_importance(clf_catboost, val_slice_x, val_slice_y.values,
                           n_repeats=4,
                           random_state=0)

                score_decreases_list.append(score_decreases)
                
            val_df_tmp = df_train.iloc[val_index].reset_index(drop=False)
            val_df_tmp['predictions'] = preds_cb
            val_df_tmp['random_seed'] = random_seed
            all_val_df_tmp.append(val_df_tmp)

    if reduce:
        return np.mean(metric_list), models, pd.concat(all_val_df_tmp).reset_index(drop=True), score_decreases_list
    else:
        return metric_list, models, pd.concat(all_val_df_tmp).reset_index(drop=True), score_decreases_list

In [32]:
# study = optuna.create_study(
#     direction="maximize"
# )
# study.optimize(optimise_catboost, timeout=60*60*6)

In [33]:
model_config_cb = ModelConfigCB(**{
     'learning_rate': 0.02606161517843435,
     'l2_leaf_reg': 18.04422276698195,
     'random_strength': 4.7069580783889995,
     'depth': 6,
     'bagging_temperature': 0.8735940473548339,
     'border_count': 256,
     'grow_policy': 'Lossguide',
     'min_data_in_leaf': 38})


kaggle_model_config = {
    'loss_function':     'Logloss',
    'iterations':        200,
    'verbose':           False,
    'random_state':      seed,
    'max_depth':         7, 
    'learning_rate':     0.06936242010150652, 
    'scale_pos_weight':  2.6149345838209532, 
    'l2_leaf_reg':       6.216113851699493, 
    'subsample':         0.6249261779711819, 
    'min_data_in_leaf':  24,
    'cat_features':      cat_cols,
}

columns_to_drop=['tbp_lv_B',
 'tbp_lv_C',
 'tbp_lv_H',
 'tbp_lv_L',
 'tbp_lv_radial_color_std_max',
 'tbp_lv_y',
 'tbp_lv_z',
 'luminance_contrast',
 'lesion_color_difference',
 'normalized_lesion_size',
 'tbp_lv_norm_border_patient_norm',
 'lesion_color_difference_patient_norm',
 'age_normalized_nevi_confidence_2_patient_norm',
 'tbp_lv_deltaA',

'size_age_interaction', 'tbp_lv_deltaB', 'oof_eva_score__cluster',
'hue_contrast_patient_norm__cluster',
'color_contrast_index_patient_norm', 'tbp_lv_Lext__cluster',
'tbp_lv_Cext_patient_norm', 'oof_eva_score_m',
'age_approx_patient_norm', 'volume_approximation_3d_patient_norm'
] 

In [34]:
scores_base, models, oof_values, fe_raw = optimise_catboost(model_config_cb.dict(), test=True, reduce=False,
    columns_to_drop=columns_to_drop,
    cat_cols=cat_cols, get_fe=False)

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric perio

In [ ]:
np.mean(scores_base)

In [ ]:
# fe_df = pd.DataFrame({
#     "feature_cols": [i for i in feature_cols if i not in columns_to_drop],
#     "mean": np.mean(np.concatenate([i['importances'] for i in fe_raw], axis=1), axis=1),
#     "std": np.std(np.concatenate([i['importances'] for i in fe_raw], axis=1), axis=1)})
# fe_df = fe_df.sort_values(by='mean', ascending=False).reset_index(drop=True)
# fe_df.head(2)